In [ ]:
### Natural Language Processing to classify genes as either provirus or phage tail using annotation description
### Recurrent Neural Network (RNN) is commonly used for Natural Language Processing. 
### RNN stores information for current features and neighboring features and is useful if prediction is at word-level 
### Long Short-Term Memory Cell (LSTM) are similar to RNNs except that hidden layer updates are replaced by memory cells which is useful for long range dependencies such as sentence structures

### scripts adapted from:
### https://www.tensorflow.org/tutorials/keras/text_classification
### https://www.tensorflow.org/tutorials/structured_data/feature_columns
### https://www.analyticsvidhya.com/blog/2020/12/understanding-text-classification-in-nlp-with-movie-review-example-example/
### https://stackoverflow.com/questions/58636087/tensorflow-valueerror-failed-to-convert-a-numpy-array-to-a-tensor-unsupporte

In [ ]:
#%pip install absl-py
#%pip install tensorflow --upgrade
#%pip install numpy --upgrade
#%pip install -U scikit-learn

In [149]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [150]:
import pandas as pd
import numpy as np
import sys
import os
import re
import string
import shutil

import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split

from tensorflow import feature_column
from tensorflow.keras import layers
from tensorflow.keras import losses
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

In [151]:
### set work directory
os.chdir("/pscratch/sd/j/jdyuzon/snakemake-bacteriophage2")

In [152]:
### load data for gene annotation descriptions **** Later include bacterial chromosome and plasmid data ****
genes_phage_tail=pd.read_csv("prediction_out/all_genes.phage_tail.bed", sep='\t')
# subset data to include ID, type, gene, annotation description
genes_phage_tail = genes_phage_tail[["ID", "type", "annotationdescription", "gene"]]
# remove NAs
genes_phage_tail=genes_phage_tail.dropna(axis=0, how="any", subset=None, inplace=False)
genes_phage_tail

/tmp/ipykernel_205871/3735015365.py:2: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  genes_phage_tail=pd.read_csv("prediction_out/all_genes.phage_tail.bed", sep='\t')


,ID,type,annotationdescription,gene
0,T6SS00897,T6SS,"type VI secretion-associated protein, ImpA family",AF361470.1_2
2,T6SS00897,T6SS,"EvpB/VC_A0108, tail sheath gpW/gp25-like domain",AF361470.1_4
3,T6SS00897,T6SS,"EvpB/VC_A0108, tail sheath gpW/gp25-like domain",AF361470.1_5
5,T6SS00897,T6SS,Predicted component of the type VI protein sec...,AF361470.1_7
6,T6SS00897,T6SS,Type VI protein secretion system component VasA,AF361470.1_8
...,...,...,...,...
103460,T6SS17506,T6SS,Predicted component of the type VI protein sec...,NZ_SRJG01000005.1_386
103466,T6SS17506,T6SS,Gp5 N-terminal OB domain,NZ_SRJG01000005.1_392
103467,T6SS17506,T6SS,"type VI secretion system effector, Hcp1 family",NZ_SRJG01000005.1_393
103468,T6SS17506,T6SS,ATP-dependent Clp protease ATP-binding subunit...,NZ_SRJG01000005.1_394


In [153]:
# some rows are duplicates where genes have the same annotation. drop duplicate rows
genes_phage_tail=genes_phage_tail.drop_duplicates()
genes_phage_tail

,ID,type,annotationdescription,gene
0,T6SS00897,T6SS,"type VI secretion-associated protein, ImpA family",AF361470.1_2
2,T6SS00897,T6SS,"EvpB/VC_A0108, tail sheath gpW/gp25-like domain",AF361470.1_4
3,T6SS00897,T6SS,"EvpB/VC_A0108, tail sheath gpW/gp25-like domain",AF361470.1_5
5,T6SS00897,T6SS,Predicted component of the type VI protein sec...,AF361470.1_7
6,T6SS00897,T6SS,Type VI protein secretion system component VasA,AF361470.1_8
...,...,...,...,...
103460,T6SS17506,T6SS,Predicted component of the type VI protein sec...,NZ_SRJG01000005.1_386
103466,T6SS17506,T6SS,Gp5 N-terminal OB domain,NZ_SRJG01000005.1_392
103467,T6SS17506,T6SS,"type VI secretion system effector, Hcp1 family",NZ_SRJG01000005.1_393
103468,T6SS17506,T6SS,ATP-dependent Clp protease ATP-binding subunit...,NZ_SRJG01000005.1_394


In [154]:
# some genes have multiple annotations 
gene = genes_phage_tail["gene"]
genes_phage_tail[gene.isin(gene[gene.duplicated()])].sort_values("gene")
# sometimes TIGER mistakes a phage for a phage-tail especially when the phage is low quality (Phage 2)
# 50 instances where a T6SS gene was reported as a Phage2 gene

,ID,type,annotationdescription,gene
86542,Phage2,Phage2,"EvpB/VC_A0108, tail sheath gpW/gp25-like domain",NC_017626.1_4543
86712,T6SS00463,T6SS,"EvpB/VC_A0108, tail sheath gpW/gp25-like domain",NC_017626.1_4543
86714,T6SS00463,T6SS,Predicted component of the type VI protein sec...,NC_017626.1_4544
86544,Phage2,Phage2,Predicted component of the type VI protein sec...,NC_017626.1_4544
86546,Phage2,Phage2,Type VI protein secretion system component VasF,NC_017626.1_4545
86716,T6SS00463,T6SS,Type VI protein secretion system component VasF,NC_017626.1_4545
86548,Phage2,Phage2,type VI secretion system peptidoglycan-associa...,NC_017626.1_4546
86718,T6SS00463,T6SS,type VI secretion system peptidoglycan-associa...,NC_017626.1_4546
86550,Phage2,Phage2,"type VI secretion system effector, Hcp1 family",NC_017626.1_4547
86720,T6SS00463,T6SS,"type VI secretion system effector, Hcp1 family",NC_017626.1_4547


In [155]:
# remove duplicates that are Phage2 but keep T6SS since these are experimentally verified
phage2dups=genes_phage_tail[gene.isin(gene[gene.duplicated()]) & (genes_phage_tail['ID']=='Phage2')].index
genes_phage_tail=genes_phage_tail.drop(phage2dups)
genes_phage_tail

,ID,type,annotationdescription,gene
0,T6SS00897,T6SS,"type VI secretion-associated protein, ImpA family",AF361470.1_2
2,T6SS00897,T6SS,"EvpB/VC_A0108, tail sheath gpW/gp25-like domain",AF361470.1_4
3,T6SS00897,T6SS,"EvpB/VC_A0108, tail sheath gpW/gp25-like domain",AF361470.1_5
5,T6SS00897,T6SS,Predicted component of the type VI protein sec...,AF361470.1_7
6,T6SS00897,T6SS,Type VI protein secretion system component VasA,AF361470.1_8
...,...,...,...,...
103460,T6SS17506,T6SS,Predicted component of the type VI protein sec...,NZ_SRJG01000005.1_386
103466,T6SS17506,T6SS,Gp5 N-terminal OB domain,NZ_SRJG01000005.1_392
103467,T6SS17506,T6SS,"type VI secretion system effector, Hcp1 family",NZ_SRJG01000005.1_393
103468,T6SS17506,T6SS,ATP-dependent Clp protease ATP-binding subunit...,NZ_SRJG01000005.1_394


In [156]:
### Looking at the distribution of the data, it is highly imbalanced
genes_phage_tail['type'].value_counts()

Phage1    34757
Phage2     2442
T6SS       2379
eCIS        751
Name: type, dtype: int64

In [157]:
### There are 1883 unique annotation descriptions
genes_phage_tail['annotationdescription'].value_counts()

Phage-related protein                                                          814
Phage-related protein, tail component                                          643
Predicted component of the type VI protein secretion system                    583
Phage terminase-like protein, large subunit, contains N-terminal HTH domain    460
Phage terminase, small subunit                                                 442
                                                                              ... 
pca operon transcription factor PcaQ                                             1
AdoMet dependent proline di-methyltransferase                                    1
NusG family protein                                                              1
RNA polymerase sigma factor RpoD, C-terminal domain                              1
Papain family cysteine protease                                                  1
Name: annotationdescription, Length: 1883, dtype: int64

In [158]:
### Create a target variable: the label types 1 and 0 correspond to provirus and phage tail
y=genes_phage_tail['type']
y=y.replace('Phage1',1)
y=y.replace('Phage2',1)
y=y.replace('T6SS',0)
y=y.replace('eCIS',0)
genes_phage_tail['target']=y
#y = int(y)
### drop un-used columns 
genes_phage_tail = genes_phage_tail.drop(columns=['ID', 'type','gene'])
genes_phage_tail

,annotationdescription,target
0,"type VI secretion-associated protein, ImpA family",0
2,"EvpB/VC_A0108, tail sheath gpW/gp25-like domain",0
3,"EvpB/VC_A0108, tail sheath gpW/gp25-like domain",0
5,Predicted component of the type VI protein sec...,0
6,Type VI protein secretion system component VasA,0
...,...,...
103460,Predicted component of the type VI protein sec...,0
103466,Gp5 N-terminal OB domain,0
103467,"type VI secretion system effector, Hcp1 family",0
103468,ATP-dependent Clp protease ATP-binding subunit...,0


In [159]:
genes_phage_tail['target'].value_counts()

1    37199
0     3130
Name: target, dtype: int64

In [160]:
### split the data to train, validation and test
train, test = train_test_split(genes_phage_tail, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

25810 train examples
6453 validation examples
8066 test examples


In [161]:
### the TextVectorization layer calls the standardization function, tokenizes the data, and vectorize (converts sequences to numbers)
### Vectorize converts tokenized data into number so that they can be input into a neural network
max_features = 10000
sequence_length = 1000

vectorize_layer = layers.TextVectorization(
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

### Make a text-only dataset (without labels), then call adapt
### vocabulary for the layer must be either supplied on construction or learned via adapt()
### Then this layer is adapted, it will analyze the dataset, determine the frequency of individual string values, and create a vocabulary from them
vectorize_layer.adapt(train['annotationdescription'])

In [162]:
### Preprocess after train validation test split to prevent leakage
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [163]:
### Double check the dataset 
def show_shapes(): # can make yours to take inputs; this'll use local variable values
    print("Expected: (num_samples, timesteps, channels)")
    print("Sequences: {}".format(Sequences.shape))
    print("Targets:   {}".format(Targets.shape)) 


def get_data(sequences, target):
    #Sequences = np.asarray(vectorize_layer(train['annotationdescription']))
    #Targets   = np.asarray(train['target'])
    Sequences = np.asarray(vectorize_layer(sequences))
    Targets   = np.asarray(target)
    show_shapes()

    Sequences = np.expand_dims(Sequences, -1)
    Targets   = np.expand_dims(Targets, -1)
    show_shapes()
    return Sequences, Targets

### Preprocess on individual datasets to prevent leakage
train_x,train_y=get_data(train['annotationdescription'],train['target'])
val_x,val_y=get_data(val['annotationdescription'],val['target'])
test_x,test_y=get_data(test['annotationdescription'],test['target'])

Expected: (num_samples, timesteps, channels)
Sequences: (25810, 1000, 1)
Targets:   (25810, 1)
Expected: (num_samples, timesteps, channels)
Sequences: (25810, 1000, 1)
Targets:   (25810, 1)
Expected: (num_samples, timesteps, channels)
Sequences: (25810, 1000, 1)
Targets:   (25810, 1)
Expected: (num_samples, timesteps, channels)
Sequences: (25810, 1000, 1)
Targets:   (25810, 1)
Expected: (num_samples, timesteps, channels)
Sequences: (25810, 1000, 1)
Targets:   (25810, 1)
Expected: (num_samples, timesteps, channels)
Sequences: (25810, 1000, 1)
Targets:   (25810, 1)


In [164]:
model = Sequential()

model.add(LSTM(128, activation='sigmoid',
               input_shape=(1000, 1), return_sequences=True))
model.add(Dropout(0.2)) # Dropout layer help prevent overfitting by randomly setting input units to 0 with the specified rate
model.add(LSTM(128, activation='sigmoid')) # 128 units: number of units is the dimension of the hidden state (or the output); defines the dimension of hidden states (or outputs) and the number of params
model.add(Dropout(0.2))
model.add(Dense(32, activation='sigmoid')) # Dense layer is set of densly connected neurons 
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

#opt = tf.keras.optimizers.Adam(lr=1e-3)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_x,train_y, epochs=10, validation_data=(val_x,val_y))


Epoch 1/3
807/807 [==============================] - 919s 1s/step - loss: 0.2769 - accuracy: 0.9227 - val_loss: 0.2877 - val_accuracy: 0.9166
Epoch 2/3
807/807 [==============================] - 899s 1s/step - loss: 0.2740 - accuracy: 0.9238 - val_loss: 0.2894 - val_accuracy: 0.9166
Epoch 3/3
807/807 [==============================] - 932s 1s/step - loss: 0.2741 - accuracy: 0.9238 - val_loss: 0.2869 - val_accuracy: 0.9166


In [128]:
loss, accuracy = model.evaluate(test_x,test_y)

807/807 [==============================] - 423s 523ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


In [ ]:
opt